In [4]:
from datasets import load_dataset
import json
import glob




/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:

ds_name = "kanhatakeyama/wizardlm8x22b-logical-math-coding-sft"

ds = load_dataset(ds_name, split="train")


def load_dict():
    en_ja_dict = {}
    for en_ja_dict_path in glob.glob("out_translated_ja/*.jsonl"):
        print("loading ", en_ja_dict_path)
        with open(en_ja_dict_path) as f:
            lines = f.readlines()
        for line in lines:
            d = json.loads(line)
            en_ja_dict[list(d.keys())[0]] = list(d.values())[0]

    return en_ja_dict


en_ja_dict = load_dict()

loading  out_translated_ja/model_20240805170939_700482.jsonl
loading  out_translated_ja/model_20240805172715_428058.jsonl
loading  out_translated_ja/model_20240805172747_227318.jsonl
loading  out_translated_ja/model_20240805171802_572384.jsonl
loading  out_translated_ja/model_20240805170338_615541.jsonl
loading  out_translated_ja/model_20240805171631_471878.jsonl
loading  out_translated_ja/en_ja_dict.jsonl
loading  out_translated_ja/model_20240805170638_473565.jsonl


In [16]:
# %%
content_list = []
for r in ds:
    translated_messages=[]
    for messages in r["messages"]:
        content=messages["content"]
        if content in en_ja_dict:
            content=en_ja_dict[content]
            print("translated")
        msg={"role":messages["role"],"content":content}
        translated_messages.append(msg)
    content_list.append({"messages":translated_messages})

translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated
translated

In [23]:
import datasets
tr_ds=datasets.Dataset.from_list(content_list)

In [26]:
tr_ds.push_to_hub("kanhatakeyama/wizardlm8x22b-logical-math-coding-sft-ja")

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.90s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/kanhatakeyama/wizardlm8x22b-logical-math-coding-sft-ja/commit/e51861f0d1d0e4a7d82de448172986e635d2d94b', commit_message='Upload dataset', commit_description='', oid='e51861f0d1d0e4a7d82de448172986e635d2d94b', pr_url=None, pr_revision=None, pr_num=None)